In [7]:
import os
import csv
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from collections import defaultdict

In [8]:
pr_file_path="C:/apache-tomcat-7.0.34/webapps/Tutorial_1/"

with open(pr_file_path+"/Transactionsdata_train.csv", "r") as f:
    reader = csv.DictReader(f, delimiter=',')
    with open(pr_file_path+"/Transactionsdata_test.csv", "w",newline='') as f_out:
        writer = csv.DictWriter(f_out, fieldnames=reader.fieldnames, delimiter=",")
        for row in reader:
            writer.writerow(row)
            
file_path = os.path.expanduser(pr_file_path+'/Transactionsdata_test.csv')

# As we're loading a custom dataset, we need to define a reader. In the
# movielens-100k dataset, each line has the following format:
# 'user item rating timestamp', separated by '\t' characters.
reader = Reader(line_format='user item rating', sep=',')

#data = Dataset.load_from_file(file_path, reader=reader)

def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# First train an SVD algorithm on the movielens dataset.
data = Dataset.load_from_file(file_path, reader=reader)
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=3)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
    
out = open(pr_file_path+'/matrixFactorizationBasedRecommendations.csv', 'w',newline='')
output=csv.writer(out)

for uid, user_ratings in top_n.items():
    output.writerow([uid, [iid for (iid, _) in user_ratings]])
    
out.close()


harsh ['Apple Macbook Pro', 'Apple Macbook Pro 16', 'Apple iPhone 11 Pro']
ninad ['Apple iPhone 11 Pro']
chirag ['Apple Macbook Pro 16', 'Apple iPhone 11', 'Apple Macbook Air']
chris ['Apple Macbook Pro 16', 'Apple Macbook Pro', 'Apple iPhone 11 Pro Max']
nishant ['Apple iPhone 11 Pro Max', 'Apple Macbook Pro 16', 'Dell Inspiron 6546']
satyaveer ['Apple Macbook Pro 16', 'Dell Inspiron 6546', 'Apple iPhone 11 Pro']
akash ['Apple Macbook Pro', 'Apple iPhone 11 Pro', 'Apple iPhone 11']
dhiraj ['Apple iPhone 11 Pro Max', 'Apple Macbook Pro', 'Dell Inspiron 6546']
jose ['Apple Macbook Pro 16', 'Apple iPhone 11 Pro Max', 'Apple iPhone 11 Pro']
